<a href="https://colab.research.google.com/github/franklinwillemen/Sherlock-Hemlock-Graph-Based-NER/blob/main/data_build_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import transformers as t
import datasets
from datasets import load_dataset

import torch
import torch_geometric
from torch_geometric.data import Data

import numpy as np
import pandas as pd

import spacy
from spacy.util import minibatch

from sklearn.preprocessing import LabelEncoder

c:\Users\Frank\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
#articles = load_dataset("maastrichtlawtech/bsard", data_files="articles_fr.csv", split="train")
#articles = pd.DataFrame(articles.to_pandas())
#articles.head()

# articles are already processed and downloadable from huggingface
df = pd.read_csv("../../local_datasets/bsard_extra/bsard_articles_extra_spacy.csv")

In [17]:
#q_train = load_dataset("maastrichtlawtech/bsard", data_files="questions_fr_train.csv", split="train").to_pandas()
#q_test = load_dataset("maastrichtlawtech/bsard", data_files="questions_fr_test.csv", split="train").to_pandas()
#q_synth = load_dataset("maastrichtlawtech/bsard", data_files="questions_fr_synthetic.csv", split="train").to_pandas()

In [28]:
df

,id,article,pos,dep,heads
0,1,Le présent Code règle une matière visée à l'ar...,"[['DET', 'ADJ', 'NOUN', 'VERB', 'DET', 'NOUN',...","[['det', 'amod', 'nsubj', 'ROOT', 'det', 'obj'...","[[2, 2, 3, 3, 5, 3, 5, 9, 9, 6, 9, 13, 13, 9, 3]]"
1,2,Le présent Code transpose en Région de Bruxell...,"[['DET', 'ADJ', 'NOUN', 'VERB', 'ADP', 'NOUN',...","[['det', 'amod', 'nsubj', 'ROOT', 'case', 'obl...","[[2, 2, 3, 3, 5, 3, 7, 5, 5, 5, 11, 3, 11, 3],..."
2,3,Le présent Code poursuit les objectifs suivant...,"[['DET', 'ADJ', 'NOUN', 'VERB', 'DET', 'NOUN',...","[['det', 'amod', 'nsubj', 'ROOT', 'det', 'obj'...","[[2, 2, 3, 3, 5, 3, 5, 3], [2, 2, 2, 4, 2, 4, ..."
3,4,"Au sens du présent Code, il faut entendre par ...","[['ADP', 'NOUN', 'ADP', 'ADJ', 'NOUN', 'PUNCT'...","[['case', 'obl:mod', 'case', 'amod', 'nmod', '...","[[1, 7, 4, 4, 1, 7, 7, 7, 7, 10, 8, 7, 13, 11,..."
4,5,"Le plan régional Air-Climat-énergie, ci-après ...","[['DET', 'NOUN', 'ADJ', 'PROPN', 'PROPN', 'PRO...","[['det', 'nsubj', 'amod', 'nmod', 'nmod', 'nmo...","[[1, 18, 1, 1, 1, 1, 1, 1, 1, 12, 12, 12, 1, 1..."
...,...,...,...,...,...
22628,22629,La ville de Bruxelles est la capitale de la Be...,"[['DET', 'NOUN', 'ADP', 'PROPN', 'AUX', 'DET',...","[['det', 'nsubj', 'case', 'nmod', 'cop', 'det'...","[[1, 6, 3, 1, 6, 6, 6, 9, 9, 6, 12, 12, 6, 14,..."
22629,22630,Le pouvoir législatif fédéral a le droit de dé...,"[['DET', 'NOUN', 'ADJ', 'ADJ', 'VERB', 'DET', ...","[['det', 'nsubj', 'amod', 'amod', 'ROOT', 'det...","[[1, 4, 1, 1, 4, 6, 4, 8, 6, 12, 12, 12, 8, 12..."
22630,22631,Aucune révision de la Constitution ne peut êtr...,"[['DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADV', ...","[['det', 'nsubj', 'case', 'det', 'nmod', 'advm...","[[1, 6, 4, 4, 1, 6, 6, 8, 6, 10, 8, 12, 10, 14..."
22631,22632,"Pendant une régence, aucun changement ne peut ...","[['ADP', 'DET', 'NOUN', 'PUNCT', 'DET', 'NOUN'...","[['case', 'det', 'obl:mod', 'punct', 'det', 'n...","[[2, 2, 7, 7, 5, 7, 7, 7, 9, 7, 12, 12, 9, 18,..."


In [52]:
# Processing using Spacy pipelines
snlp = spacy.load("fr_dep_news_trf")

def extract_pos_dep_head(docs):
    pos, dep, heads = [], [], []
    
    for doc in docs:
        pos_doc, dep_doc, heads_doc = [], [], []
        for sent in doc.sents:
            pos_sent = [token.pos_ for token in sent]
            dep_sent = [token.dep_ for token in sent]
            heads_sent = [token.head.i - sent.start for token in sent]
            pos_doc.append(pos_sent)
            dep_doc.append(dep_sent)
            heads_doc.append(heads_sent)
        pos.append(pos_doc)
        dep.append(dep_doc)
        heads.append(heads_doc)
    
    return pos, dep, heads

def process_with_spacy(df, nlp, batch_size=32):
    pos, dep, heads = [], [], []
    
    for i in range(0, len(df), batch_size):
        batch_text = df['article'][i:i+batch_size].tolist()
        
        # Process batch with Spacy
        docs = list(nlp.pipe(batch_text))
        
        # Extract POS, DEP, HEADS
        pos_batch, dep_batch, heads_batch = extract_pos_dep_head(docs)
        pos.extend(pos_batch)
        dep.extend(dep_batch)
        heads.extend(heads_batch)
        
    df.loc[:, 'pos'] = pd.Series(pos)
    df.loc[:, 'dep'] = pd.Series(dep)
    df.loc[:, 'heads'] = pd.Series(heads)
    
    return df

def tokenize_sentences(df, nlp, batch_size=32):
    tokens = []
    
    for i in range(0, len(df), batch_size):
        batch_text = df['article'][i:i+batch_size].tolist()
        
        # Process batch with Spacy
        docs = list(nlp.pipe(batch_text))
        
        # Tokenize sentences
        for doc in docs:
            doc_tokens = []
            for sent in doc.sents:
                sent_tokens = [token.text for token in sent]
                doc_tokens.append(sent_tokens)
            tokens.append(doc_tokens)
    
    df['tokens'] = tokens
    
    return df

In [22]:
df = df.drop(["section","reference", "subsection", "code","act","chapter", "book", "word_count", "sentence_count"], axis=1)

In [25]:
df.to_csv("../../local_datasets/bsard_extra/bsard_articles_preprocessed.csv", index=False)

In [32]:
q_train["article_ids"]

0      22225,22226,22227,22228,22229,22230,22231,2223...
1                                         5853,5854,5855
2                          1096,1097,1098,1108,1109,1110
3              12012,12030,12031,12032,12033,12034,12035
4      21114,21115,21116,21117,21118,21119,21120,2112...
                             ...                        
881              1321,1322,1323,1324,1325,1326,1327,1328
882                                                 1381
883                                                 2494
884                                                 1387
885    5861,5862,5863,5864,5865,5866,5867,5868,5869,5...
Name: article_ids, Length: 886, dtype: object

In [33]:
q_test["article_ids"]

0                                         12024
1                                       947,948
2                                2041,2119,2138
3                      5562,5563,5564,5565,5566
4      5561,5562,5563,5564,5565,5566,5567,12124
                         ...                   
217                              2328,2329,2330
218                                        2127
219                                        1073
220                                         857
221                                   6531,6598
Name: article_ids, Length: 222, dtype: object

In [38]:
q_synth.iloc[10000]

id                                  11195
question       ce qui a causé la violence
article_ids                          2001
Name: 10000, dtype: object

In [44]:
df[df["id"] == 2001]["article"].values

array(["Il y a violence, lorsqu'elle est de nature à faire impression sur une personne raisonnable, et qu'elle peut lui inspirer la crainte d'exposer sa personne ou sa fortune à un mal considérable et présent.On a égard, en cette matière, à l'âge, au sexe et à la condition des personnes."],
      dtype=object)

In [45]:
df_ = pd.read_csv("../../local_datasets/bsard_extra/bsard_articles_extra.csv")

In [47]:
df_

,Unnamed: 0,id,reference,article,law_type,code,book,part,act,chapter,section,subsection,description,preprocessed_article,word_count,sentence_count
0,0,1,"Art. 1.1.1, Code Bruxellois de l'Air, du Clima...",Le présent Code règle une matière visée à l'ar...,regional,"Code Bruxellois de l'Air, du Climat et de la M...",Dispositions communes,NaN,Généralités,NaN,NaN,NaN,"Dispositions communes, Généralités",pr sent code r gle mati re vi e article consti...,11,1
1,1,2,"Art. 1.1.2, Code Bruxellois de l'Air, du Clima...",Le présent Code transpose en Région de Bruxell...,regional,"Code Bruxellois de l'Air, du Climat et de la M...",Dispositions communes,NaN,Généralités,NaN,NaN,NaN,"Dispositions communes, Généralités",pr sent code transpose r gion bruxelles capita...,248,4
2,2,3,"Art. 1.2.1, Code Bruxellois de l'Air, du Clima...",Le présent Code poursuit les objectifs suivant...,regional,"Code Bruxellois de l'Air, du Climat et de la M...",Dispositions communes,NaN,Objectifs,NaN,NaN,NaN,"Dispositions communes, Objectifs",pr sent code poursuit objectifs suivants int g...,136,2
3,3,4,"Art. 1.3.1, Code Bruxellois de l'Air, du Clima...","Au sens du présent Code, il faut entendre par ...",regional,"Code Bruxellois de l'Air, du Climat et de la M...",Dispositions communes,NaN,Définitions,NaN,NaN,NaN,"Dispositions communes, Définitions",sen pr sent code faut entendre r gion r gion b...,293,1
4,4,5,"Art. 1.4.1, Code Bruxellois de l'Air, du Clima...","Le plan régional Air-Climat-énergie, ci-après ...",regional,"Code Bruxellois de l'Air, du Climat et de la M...",Dispositions communes,NaN,Plan régional air-climat-énergie,Contenu et portée du plan,NaN,NaN,"Dispositions communes, Plan régional air-clima...",plan r gional air climat nergie ci apr nomm pl...,77,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22628,22628,22629,"Art. 194, La Constitution (Titre VII)",La ville de Bruxelles est la capitale de la Be...,federal,La Constitution,NaN,NaN,Dispositions generales.,NaN,NaN,NaN,Dispositions generales,ville bruxelles capitale belgique si ge gouver...,9,1
22629,22629,22630,"Art. 195, La Constitution (Titre VIII)",Le pouvoir législatif fédéral a le droit de dé...,federal,La Constitution,NaN,NaN,De la revision de la constitution.,NaN,NaN,NaN,De la revision de la constitution,pouvoir gislatif f ral a droit clarer a lieu r...,753,8
22630,22630,22631,"Art. 196, La Constitution (Titre VIII)",Aucune révision de la Constitution ne peut êtr...,federal,La Constitution,NaN,NaN,De la revision de la constitution.,NaN,NaN,NaN,De la revision de la constitution,aucune r vision constitution peut tre engag e ...,23,1
22631,22631,22632,"Art. 197, La Constitution (Titre VIII)","Pendant une régence, aucun changement ne peut ...",federal,La Constitution,NaN,NaN,De la revision de la constitution.,NaN,NaN,NaN,De la revision de la constitution,pendant r gence aucun changement peut tre appo...,15,1
